In [34]:
import nltk
import numpy as np
import pandas as pd
from textblob import TextBlob
import re

from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer 

from nltk.tokenize import TweetTokenizer

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

import random
import string
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer        
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, classification_report

from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Load Data

In [35]:
df = pd.read_csv("K8 Reviews v0.2.csv")

In [36]:
df.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [37]:
df.sentiment.value_counts(normalize=True)

0    0.52552
1    0.47448
Name: sentiment, dtype: float64

In [38]:
df.shape

(14675, 2)

# Convert the review to lower case

In [39]:
reviewCol = df['review'].str.lower()

In [40]:
reviewCol

0                   good but need updates and improvements
1        worst mobile i have bought ever, battery is dr...
2        when i will get my 10% cash back.... its alrea...
3                                                     good
4        the worst phone everthey have changed the last...
5        only i'm telling don't buyi'm totally disappoi...
6        phone is awesome. but while charging, it heats...
7                          the battery level has worn down
8        it's over hitting problems...and phone hanging...
9        a lot of glitches dont buy this thing better g...
10                                                   wrost
11       good phone but charger not working / damage wi...
12       don't purchase this item, it is so much of hea...
13       i have faced the battery problem and motherboa...
14       very good phone slim good battry backup good s...
15                                headset is not available
16       every time automatic on and off so kindly sugg.

In [41]:
df['reviewLower'] = pd.Series(reviewCol, index=df.index)

# Remove stop words and other words except noun

In [42]:
nouns = []
for item in reviewCol:
    eachRow = []
    zen = TextBlob(re.sub("/b[0-9]/b","", item.replace(".", " ")))
    for sent in zen.sentences:
        for word in sent.tags:
            if re.match('NP*', word[1]):
                if not word[0] in stopwords.words('english'):
                    eachRow.append(word[0])
    nouns.append(list(set(eachRow)))
    print(list(set(eachRow)))

['updates', 'improvements']
['charger', 'idle', 'amazon', 'mobile', 'hell', 'lie', 'uses', 'hours', 'battery', 'lenove', 'backup']
['%', 'cash']
[]
['everthey', 'phone', 'amazon', 'problem']
['camerawaste', 'money']
['phone', 'reason', 'k8']
['battery', 'level']
['note', 'years', 'phone', 'station', 'ahmedabad', 'lenovo', 'problems', 'hanging']
['options', 'glitches', 'thing', 'lot']
['wrost']
['phone', 'months', 'charger', 'damage']
['battery', 'life', 'item']
['months', 'mobile', 'problem', 'life', 'battery', 'motherboard']
['battry', 'screen', 'slim', 'phone', 'backup']
['headset']
['time']
['seal', 'deal', 'range', 'amazon', 'comparison', 'mobile', 'specification', 'phone', 'product', 'card', 'prize', 'credit']
['battery', 'solutions', 'life']
['smartphone']
[]
['speaker', 'phone', 'problem', 'galery']
['speed', 'battery', 'features', 'camera']
['product']
['phone', 'battery', 'product', 'os', 'lenovo', 'camera']
['screen', 'options', 'mobile', 'hotspot', 'cast', 'call', 'option']


['performance']
['mistake', 'battery', 'camera', 'phone']
['battery', 'backup']
['update', 'senser', 'screen', 'deapth', 'time', 'side', 'problem', '😎😎', 'phone', 'battery', 'print', 'opinion', 'buttons', 'gorilla', 'glass', 'mode', 'camera']
['works', 'sim', 'battery', 'data']
['age', 'clarity', 'quality', 'group', 'screen', 'battery', 'product', 'backup', 'camera']
['use', 'amazon', 'thank', 'phone', 'price', 'battery', 'depth', 'thing', 'days', 'mode', 'camera']
['system', 'time', 'product', 'network', 'internet', 'lte']
['bit', 'bugs', 'smartphone', 'cost']
['product']
['traps', 'idiots', 'service', 'crap', 'care', 'center', 'phone', 'issues', 'response', 'customer']
[]
['gold', 'phone', 'variant']
[]
[]
['performance', 'battery', 'charging', 'time']
['phone', 'work', 'touchscreen']
['features', 'rs']
['performance']
['amazon', 'money', 'service']
['purchase', 'months', 'touch']
['%', 'product', 'thanks']
['amazon', 'product', 'lenovo', 'issue']
['technician', 'phone', 'issues', 'a

['battery', 'time', 'problem', 'please']
['experience', 'hour', 'day', 'quality', 'time', 'speed', 'phone', 'battery', 'installation', 'backup', 'camera', 'charge']
['performance', 'cover', 'everything', 'effect', 'phone', 'product', 'case']
['lenovo']
[]
['mobile', 'speaker', 'company', 'lenovo', 'camera']
['smartphone', 'range']
['work']
['fine']
['features', 'camera', 'mi']
[]
['heating', 'product', 'mobile', 'problem']
['battery', 'backup', 'thing']
['run', 'phone', 'weeks', 'issue']
['performance']
['heating', 'update', 'mobile', 'review', 'phone', 'problems', 'camera']
[]
[]
['mobile', 'average']
[]
['call', 'disconnect', 'volte', 'amazon', 'phone', 'lenovo', 'wifi']
['drop', 'charging', 'problem', 'needs']
['focus', 'software', 'defects', 'contact', 'interface', 'modification', 'management', 'auto', 'hardware', 'brightness', 'customer', 'conference', 'option', 'functions', 'system', 'lack', 'support', 'product', 'lenovo', 'call', 'gesture']
['time']
['combination', 'camera']
['p

['overheating', 'month', 'battery', 'hours', 'trouble', 'backup']
['mobile', 'issue']
['quality', 'occurs', 'screen', 'mobile', 'gorilla']
['features', 'battery', 'hrs', 'backup', 'camera']
['cpu']
['lover', 'screen', 'helio', 'thanks', 'deca', 'phone', 'effect', 'uses', 'point', 'front', 'problems', 'mah', 'bokeh', 'processor', 'roms', 'depth', 'rear', 'object', 'speaker', 'user', 'verdict', 'camera', 'core', 'g5+', 'performance', 'mediatek', 'use', 'day', 'clarity', 'fan', 'honor', 'choice', 'options', 'moto', 'price', 'battery', 'cameras', 'android', 'earphone', 'market', 'background', 'note', 'stock', 'display', 'features', 'custom', 'ghz', 'lenovo', 'x23', 'edges', 'things']
['graphics']
[]
['bekar', 'ha', 'lo', 'mat']
['product']
['headphone', 'months', 'pack', 'booklet', 'phone', 'warranty', 'earphone']
['service', 'center', 'problem', 'month', 'phone', 'response', 'bettery']
['security', 'fingerprint']
['specs', 'phone', 'problems']
['product']
['mistake', 'camera', 'problem']


['phn']
['phone', 'charger', 'chargerthe', 'hours']
['updates', 'k8', 'awesome', 'battery', 'please', 'lenovo', 'plz', 'backup', 'setting', 'camera']
['mobile', 'speed', 'bluetooth', 'settings', 'screenshot', 'discharging', 'dial', 'cost']
['prdt']
['quality', 'wothy', 'mobile', 'charging', 'standardsvery', 'price', 'battery', 'backup', 'camera']
['beast', 'way', 'software', 'selfie', 'depth', 'battery', 'chipset', 'camera', 'sensing']
['cast', 'screen', 'feature', 'product']
['yes']
['rs', 'screen', 'money', 'delicate', 'waste']
['screen', 'a1', 'phone', 'terms', 'hand', 'phones', 'depth', 'design', 'camera', 'use', 'moto', 'honor', 'video', 'app', 'price', 'android', 'mode', 'background', 'bugs', 'perfect', 'update', 'updates', 'stock', 'pictures', 'segment', 'crashes', 'motion', 'bonus', 'lenovo']
['usage', 'worth']
['google', 'manufacturers', 'difference', 'experience', 'edition', 'competiror', 'equilizer', 'songs', 'speakers', 'person', 'front', "'with", 'music', 'surround', 'mobi

...

['bloatwares', 'buyers', 'couple', 'return', 'uis', 'phone', 'point', 'person', 'card', 'facie', 'music', 'months', 'fact', 'something', 'receipt', 'redmi', 'review', 'piece', 'feature', 'os', 'usability', 'usage', 'tray', 'xiaomi', 'acceptance', 'procedure', 'perspective', 'anyday', 'nano', 'camera', 'claim', 'eui', 'mistake', 'miui', "'ease", 'hence', 'hype', 'day', 'intelligence', 'sim', 'etc', 'logic', 'infact', 'touchwiz', 'observation', 'devoid', 'memory', 'snaps', 'android', 'customer', 'bouquet', 'policy', 'attempt', 'max', 'dslrs', 'mentioning', 'question', 'slots', 'note', 'otherwise', 'amazon', 'stock', 'features', 'nothing', 'sort', 'support', 'product', 'layers', 'custom', 'galaxy', 'lag/slowness', 'lenovo', 'slot', 'days', 'tread']
['charging', 'month', 'battery', 'hrs', 'mns']
['battery', 'lol', 'mah', 'backup', 'camera']
['speed', 'phone', 'camera', 'quality']
['effect', 'phone', 'sone', 'usp', 'music', 'depth', 'rear', 'package', 'camera', 'budget', 'smartphone', 'love

['hindi-', 'experience', 'nice', 'right', 'chrome', 'condition', 'life', 'share', 'range', 'processor', 'mutli', 'language', 'speaker', 'internet', 'basis', 'sensitivity', 'map', 'note', 'capturing', 'weeks', 'image', 'shutter', 'mode-', 'qr', 'rom-', 'savings', 'flash', 'music', 'edges', 'everything', 'bit-', 'moment', 'feature', 'words', 'ram', 'week', 'performance', 'shots', 'clarity', 'video', 'average', 'price', 'battery', 'clicks', 'memory', 'details', 'sound', 'display', 'size', 'speech', 'notification', 'bank', 'images', 'google', 'mp', 'couple', 'pad', 'compare', 'fingerprint', 'barcode', 'phone-', 'applications', 'review', 'rounder', 'speed', 'key', 'gif', 'speciality', 'gaming', 'photos', 'solution', 'light', 'file', 'office', 'cons', 'prosser', 'gb', 'rom', 'screen', 'sensor', 'photography', 'photograph', 'phone', 'speakers', 'games', 'bit', 'typing', 'depth', 'user', 'calls', 'code', 'camera', 'quality', 'k8', 'selfie', 'part-', 'htc', 'flashlight', 'mode', 'key-', 'featur

['price']
['class', 'hour', 'phone', 'battery', 'charge']
[]
['range', 'video', 'gallery', 'phone', 'application', 'player', 'music']
['piece', 'duplicate']
['value', 'display', 'money', 'speed', 'spent', 'phone']
['exchange', 'system', 'mobile', 'time', 'return', 'purchase', 'product', 'policy']
['phoneall', 'middle', 'features']
['replacement', 'need']
['phone', 'features']
['replacement', 'ek', 'gye', 'purchase', 'phone', 'baad', 'bekaarpehle', 'bhi', 'charger', 'lg', 'itna', 'kri', 'din', 'complaint', 'pehla', 'krna', 'bekaaar', 'mera', 'dum', 'kiyaa', 'tha', 'h', 'product', 'ye']
['core', 'everything', 'touch', 'range', 'mp', 'display', 'excellent', 'deca', 'phone', 'battery', 'rear', 'front', 'backup', 'mode', 'function', 'music', 'camera']
['hai', 'handset', 'ho', 'plz', 'charger', 'gaya']
['hangs', 'processor', 'speed', 'phone', 'product']
['products']
['software', 'flash', 'cover6', 'basics', 'steaming', 'door2', 'charger', 'speed', 'os', 'network', 'caller', 'list6', 'need', 

['product']
['mobile', 'camera']
['phone', 'right', 'choose']
['pin', 'processor', 'sim', 'features', 'handsfree', 'memory', 'x22', 'card', 'camera']
['phone', 'battery', 'price', 'front', 'backup', 'camera']
['fine']
['superb', 'mobile']
['seconds', 'phone', 'stick', 'range', 'lacks', 'phones', 'surfingi', 'network', 'fire', 'looks', 'price', 'wifi', 'note', 'volte', 'irritating', '-stock', 'videos', 'issues', 'signals']
['picture', 'quality', 'range', 'mobile', 'image', 'price', 'buy', 'camera']
[]
['phone', 'battery', 'camera']
['design']
['one', 'ui', 'configuration']
['problem', 'users']
['delivery', 'thanks']
['expectations', 'product']
['phone', 'nice']
['bodykiller', 'budget', 'features', 'priceprosstock', 'processerkiller', 'effect']
['china', 'product', 'plz']
['battery', 'method', 'speaker']
[]
['value', 'service', 'buyback', 'amazon', 'concern', 'money', 'delivery', 'battery', 'amount']
[]
['screen', 'phone', 'time', 'camera']
['lenovo', 'product']
['model', 'heating', 'pro

['recording', 'sensor', 'e', 'thanks', 'problems', 'speedometer', 'percentage', 'speed', 'auto', 'try', 'etc', 'k8', 'battery', 'dialing', 'max', 'touch', 'theater', 'note', 'sound', 'features', 'center', 'call', 'notification']
['📞', 'life']
['flash', 'cores', 'product', 'specifications', 'slot', 'camera']
['phone', 'feature']
['phone', 'camera']
['phone']
['phone', 'turbo', 'time']
['phone', 'battery', 'issues']
['guys', 'beast', 'way', 'note', 'peranoma', 'device', 'display', 'etc', 'price', 'cameras', 'motion']
['lte', 'connectedit']
['phone', 'wise', 'camera']
['offer', 'range', 'sim', 'features', 'fps', 'phones', 'phone', 'flash', 'battery', 'videos', 'color', 'key', 'tray', 'card', 'zoom', 'music', 'camera']
['k8', 'bill', 'note']
['backup', 'issue', 'product']
['nd', 'lifebattery', 'life', 'phone', 'battery']
['battery', 'rest']
['signal', 'sim', 'phone', 'battery', 'iam']
['phone']
['phone', 'package']
['ate', 'phone', 'battery', '%']
['bit', 'product']
['phone', 'battery', 'd

In [43]:
nouns

[['updates', 'improvements'],
 ['charger',
  'idle',
  'amazon',
  'mobile',
  'hell',
  'lie',
  'uses',
  'hours',
  'battery',
  'lenove',
  'backup'],
 ['%', 'cash'],
 [],
 ['everthey', 'phone', 'amazon', 'problem'],
 ['camerawaste', 'money'],
 ['phone', 'reason', 'k8'],
 ['battery', 'level'],
 ['note',
  'years',
  'phone',
  'station',
  'ahmedabad',
  'lenovo',
  'problems',
  'hanging'],
 ['options', 'glitches', 'thing', 'lot'],
 ['wrost'],
 ['phone', 'months', 'charger', 'damage'],
 ['battery', 'life', 'item'],
 ['months', 'mobile', 'problem', 'life', 'battery', 'motherboard'],
 ['battry', 'screen', 'slim', 'phone', 'backup'],
 ['headset'],
 ['time'],
 ['seal',
  'deal',
  'range',
  'amazon',
  'comparison',
  'mobile',
  'specification',
  'phone',
  'product',
  'card',
  'prize',
  'credit'],
 ['battery', 'solutions', 'life'],
 ['smartphone'],
 [],
 ['speaker', 'phone', 'problem', 'galery'],
 ['speed', 'battery', 'features', 'camera'],
 ['product'],
 ['phone', 'battery', '

In [44]:
df['nouns'] = pd.Series(nouns, index=df.index)

In [45]:
df

,sentiment,review,reviewLower,nouns
0,1,Good but need updates and improvements,good but need updates and improvements,"[updates, improvements]"
1,0,"Worst mobile i have bought ever, Battery is dr...","worst mobile i have bought ever, battery is dr...","[charger, idle, amazon, mobile, hell, lie, use..."
2,1,when I will get my 10% cash back.... its alrea...,when i will get my 10% cash back.... its alrea...,"[%, cash]"
3,1,Good,good,[]
4,0,The worst phone everThey have changed the last...,the worst phone everthey have changed the last...,"[everthey, phone, amazon, problem]"
5,0,Only I'm telling don't buyI'm totally disappoi...,only i'm telling don't buyi'm totally disappoi...,"[camerawaste, money]"
6,1,"Phone is awesome. But while charging, it heats...","phone is awesome. but while charging, it heats...","[phone, reason, k8]"
7,0,The battery level has worn down,the battery level has worn down,"[battery, level]"
8,0,It's over hitting problems...and phone hanging...,it's over hitting problems...and phone hanging...,"[note, years, phone, station, ahmedabad, lenov..."
9,0,A lot of glitches dont buy this thing better g...,a lot of glitches dont buy this thing better g...,"[options, glitches, thing, lot]"


# Lemmatise the words

In [46]:
lemma = []

for noun in nouns:
    lemmaWords = []
    for item in noun:
        lemmaWords.append(lemmatizer.lemmatize(item, 'n'))

    lemma.append(list(set(lemmaWords)))

lemma        

[['improvement', 'update'],
 ['idle',
  'hour',
  'us',
  'amazon',
  'mobile',
  'hell',
  'lie',
  'battery',
  'backup',
  'lenove',
  'charger'],
 ['%', 'cash'],
 [],
 ['everthey', 'phone', 'amazon', 'problem'],
 ['camerawaste', 'money'],
 ['phone', 'reason', 'k8'],
 ['battery', 'level'],
 ['note',
  'year',
  'problem',
  'phone',
  'ahmedabad',
  'lenovo',
  'station',
  'hanging'],
 ['glitch', 'option', 'thing', 'lot'],
 ['wrost'],
 ['phone', 'charger', 'damage', 'month'],
 ['battery', 'life', 'item'],
 ['mobile', 'problem', 'life', 'month', 'battery', 'motherboard'],
 ['battry', 'screen', 'slim', 'phone', 'backup'],
 ['headset'],
 ['time'],
 ['seal',
  'deal',
  'range',
  'amazon',
  'comparison',
  'mobile',
  'specification',
  'phone',
  'product',
  'card',
  'prize',
  'credit'],
 ['battery', 'life', 'solution'],
 ['smartphone'],
 [],
 ['phone', 'problem', 'galery', 'speaker'],
 ['feature', 'battery', 'camera', 'speed'],
 ['product'],
 ['phone', 'battery', 'product', 'len

In [47]:
df['lemma'] = pd.Series(lemma, index=df.index)

In [48]:
df

,sentiment,review,reviewLower,nouns,lemma
0,1,Good but need updates and improvements,good but need updates and improvements,"[updates, improvements]","[improvement, update]"
1,0,"Worst mobile i have bought ever, Battery is dr...","worst mobile i have bought ever, battery is dr...","[charger, idle, amazon, mobile, hell, lie, use...","[idle, hour, us, amazon, mobile, hell, lie, ba..."
2,1,when I will get my 10% cash back.... its alrea...,when i will get my 10% cash back.... its alrea...,"[%, cash]","[%, cash]"
3,1,Good,good,[],[]
4,0,The worst phone everThey have changed the last...,the worst phone everthey have changed the last...,"[everthey, phone, amazon, problem]","[everthey, phone, amazon, problem]"
5,0,Only I'm telling don't buyI'm totally disappoi...,only i'm telling don't buyi'm totally disappoi...,"[camerawaste, money]","[camerawaste, money]"
6,1,"Phone is awesome. But while charging, it heats...","phone is awesome. but while charging, it heats...","[phone, reason, k8]","[phone, reason, k8]"
7,0,The battery level has worn down,the battery level has worn down,"[battery, level]","[battery, level]"
8,0,It's over hitting problems...and phone hanging...,it's over hitting problems...and phone hanging...,"[note, years, phone, station, ahmedabad, lenov...","[note, year, problem, phone, ahmedabad, lenovo..."
9,0,A lot of glitches dont buy this thing better g...,a lot of glitches dont buy this thing better g...,"[options, glitches, thing, lot]","[glitch, option, thing, lot]"


In [49]:
df.drop(['review', 'reviewLower', 'nouns'], axis='columns', inplace=True)

In [50]:
df

,sentiment,lemma
0,1,"[improvement, update]"
1,0,"[idle, hour, us, amazon, mobile, hell, lie, ba..."
2,1,"[%, cash]"
3,1,[]
4,0,"[everthey, phone, amazon, problem]"
5,0,"[camerawaste, money]"
6,1,"[phone, reason, k8]"
7,0,"[battery, level]"
8,0,"[note, year, problem, phone, ahmedabad, lenovo..."
9,0,"[glitch, option, thing, lot]"


In [51]:
Words = []
review = []

for ind in df.index: 
    if len(df['lemma'][ind]) != 0:
        Words.append(df['lemma'][ind])
        review.append(df['sentiment'][ind])

In [52]:
Words

[['improvement', 'update'],
 ['idle',
  'hour',
  'us',
  'amazon',
  'mobile',
  'hell',
  'lie',
  'battery',
  'backup',
  'lenove',
  'charger'],
 ['%', 'cash'],
 ['everthey', 'phone', 'amazon', 'problem'],
 ['camerawaste', 'money'],
 ['phone', 'reason', 'k8'],
 ['battery', 'level'],
 ['note',
  'year',
  'problem',
  'phone',
  'ahmedabad',
  'lenovo',
  'station',
  'hanging'],
 ['glitch', 'option', 'thing', 'lot'],
 ['wrost'],
 ['phone', 'charger', 'damage', 'month'],
 ['battery', 'life', 'item'],
 ['mobile', 'problem', 'life', 'month', 'battery', 'motherboard'],
 ['battry', 'screen', 'slim', 'phone', 'backup'],
 ['headset'],
 ['time'],
 ['seal',
  'deal',
  'range',
  'amazon',
  'comparison',
  'mobile',
  'specification',
  'phone',
  'product',
  'card',
  'prize',
  'credit'],
 ['battery', 'life', 'solution'],
 ['smartphone'],
 ['phone', 'problem', 'galery', 'speaker'],
 ['feature', 'battery', 'camera', 'speed'],
 ['product'],
 ['phone', 'battery', 'product', 'lenovo', 'o',

In [53]:
# Create Dictionary
id2word = corpora.Dictionary(Words)

In [54]:
# Create Corpus
texts = Words

In [55]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [56]:
# View
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [57]:
id2word[0]

'improvement'

In [58]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('improvement', 1), ('update', 1)]]

In [296]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [297]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.141*"money" + 0.107*"backup" + 0.081*"range" + 0.047*"value" + '
  '0.045*"superb" + 0.043*"ram" + 0.033*"glass" + 0.026*"key" + 0.024*"h" + '
  '0.021*"excellent"'),
 (1,
  '0.130*"note" + 0.084*"k8" + 0.070*"network" + 0.029*"handset" + '
  '0.026*"work" + 0.024*"replacement" + 0.017*"worth" + 0.017*"piece" + '
  '0.016*"headphone" + 0.014*"hai"'),
 (2,
  '0.058*"heat" + 0.052*"display" + 0.047*"life" + 0.043*"processor" + '
  '0.041*"use" + 0.037*"sim" + 0.037*"lot" + 0.033*"music" + 0.027*"budget" + '
  '0.026*"bit"'),
 (3,
  '0.255*"mobile" + 0.067*"update" + 0.063*"charger" + 0.055*"software" + '
  '0.051*"delivery" + 0.042*"turbo" + 0.039*"mark" + 0.030*"system" + '
  '0.026*"need" + 0.025*"expectation"'),
 (4,
  '0.100*"screen" + 0.068*"option" + 0.059*"everything" + 0.037*"support" + '
  '0.037*"card" + 0.035*"front" + 0.034*"box" + 0.033*"model" + 0.031*"light" '
  '+ 0.029*"user"'),
 (5,
  '0.155*"quality" + 0.110*"price" + 0.048*"hour" + 0.041*"speaker" + '
  '0.0

In [298]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=Words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.2896638668122815

Coherence Score:  0.4638450715470822


In [299]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\saaim\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.446830  0.028606       1        1  26.893021
9     -0.015245 -0.415766       2        1  11.289515
5     -0.030363  0.092578       3        1  11.190413
7     -0.033770  0.054919       4        1   9.405170
2     -0.049648  0.049595       5        1   9.275675
1     -0.057278  0.047061       6        1   8.318034
3     -0.058949  0.037772       7        1   6.401709
8     -0.066584  0.035513       8        1   6.107342
0     -0.067251  0.035295       9        1   5.638198
4     -0.067742  0.034428      10        1   5.480916, topic_info=     Category         Freq         Term        Total  loglift  logprob
16    Default  5321.000000        phone  5321.000000  30.0000  30.0000
57    Default  2874.000000       camera  2874.000000  29.0000  29.0000
49    Default  2072.000000      product  2072.000000  28.0000  28.0000
4     Default  2899.000000      battery  2899.000000  27.0000  27.0000
11    Default  1250.000000       mobile  1250.000000  26.0000  26.0000
97    Default  1332.000000      quality  1332.000000  25.0000  25.0000
43    Default   888.000000         time   888.000000  24.0000  24.0000
67    Default   946.000000        price   946.000000  23.0000  23.0000
26    Default   831.000000         note   831.000000  22.0000  22.0000
81    Default   888.000000  performance   888.000000  21.0000  21.0000
17    Default  1480.000000      problem  1480.000000  20.0000  20.0000
58    Default   788.000000      feature   788.000000  19.0000  19.0000
19    Default   608.000000        money   608.000000  18.0000  18.0000
73    Default  1313.000000        issue  1313.000000  17.0000  17.0000
3     Default   462.000000       backup   462.000000  16.0000  16.0000
20    Default   533.000000           k8   533.000000  15.0000  15.0000
40    Default   422.000000       screen   422.000000  14.0000  14.0000
91    Default   906.000000          day   906.000000  13.0000  13.0000
105   Default   443.000000      network   443.000000  12.0000  12.0000
50    Default   348.000000        range   348.000000  11.0000  11.0000
842   Default   411.000000         heat   411.000000  10.0000  10.0000
61    Default   435.000000         call   435.000000   9.0000   9.0000
25    Default   731.000000       lenovo   731.000000   8.0000   8.0000
1     Default   328.000000       update   328.000000   7.0000   7.0000
7     Default   410.000000         hour   410.000000   6.0000   6.0000
129   Default   319.000000       device   319.000000   5.0000   5.0000
75    Default   687.000000      heating   687.000000   4.0000   4.0000
66    Default   367.000000      display   367.000000   3.0000   3.0000
5     Default   311.000000      charger   311.000000   2.0000   2.0000
31    Default   286.000000       option   286.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
40    Topic10   421.252930       screen   422.170746   2.9017  -2.2992
31    Topic10   285.943420       option   286.861237   2.9007  -2.6866
492   Topic10   249.129364   everything   250.047241   2.9002  -2.8244
257   Topic10   155.700211      support   156.618088   2.8980  -3.2945
44    Topic10   155.070694         card   155.988571   2.8980  -3.2985
489   Topic10   145.540176        front   146.458008   2.8976  -3.3620
453   Topic10   144.196442          box   145.114258   2.8976  -3.3712
214   Topic10   139.458282        model   140.376114   2.8973  -3.4047
474   Topic10   130.835434        light   131.753296   2.8969  -3.4685
464   Topic10   123.771927         user   124.689789   2.8965  -3.5240
62    Topic10    97.543350         cast    98.461113   2.8945  -3.7621
1366  Topic10    84.635803      message    85.553696   2.8931  -3.9041
138   Topic10    84.000984       people    84.918808   2.8930  -3.9116
607   Topic10    59.808502          gud    60.726315   2.8887  -4.2513
463   Topic10    53.503395         team    